# 02- Data Cleaning & Basic Preprocessing

Bu notebook'ta, EDA aşamasında tespit ettiğimiz temel veri kalitesi problemlerini 
adım adım temizleyeceğiz:

1. Ham veriyi yüklemek
2. Gereksiz ID kolonunu kaldırmak
3. `age` değişkenindeki hatalı 0 değerini düzeltmek
4. `MonthlyIncome` ve `NumberOfDependents` için eksik değerleri doldurmak
5. Delinquency değişkenlerindeki (30–59, 60–89, 90+) uç değerleri sınırlamak
6. `age` değişkeninde 95+ yaş değerleri için üst sınır (capping) uygulamak
7. Temiz veriyi kaydetmek

Amaç, feature engineering ve modelleme aşamalarında kullanacağımız, tutarlı ve güvenilir bir eğitim veri seti elde etmek.


## 1. Kütüphaneler ve Dosya Yolları

Önce temel kütüphaneleri ve veri dosyalarının yolunu tanımlayacağız. 
Path yönetimi için `config.py` kullanarak, hem notebook’larda hem de script'lerde 
aynı dosya yollarını yeniden kullanabiliriz.


In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))


In [2]:
import pandas as pd
from src.config import RAW_TRAIN, CLEAN_TRAIN

pd.set_option("display.max_columns", None)

RAW_TRAIN, CLEAN_TRAIN

(WindowsPath('C:/Users/YAĞMUR/Masaüstü/credit-risk-model/data/cs-training.csv'),
 WindowsPath('C:/Users/YAĞMUR/Masaüstü/credit-risk-model/data/cs-training-clean.csv'))

## 2. Ham Verinin Yüklenmesi

EDA'da analiz ettiğimiz eğitim verisini (cs-training.csv) burada ham haliyle yeniden yüklüyoruz. 
Temizlik adımlarının tamamı bu ham veri üzerinde uygulanacak.

In [3]:
df_raw = pd.read_csv(RAW_TRAIN)
df_raw.head()


,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [4]:
df_raw.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 12 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   Unnamed: 0                            150000 non-null  int64  
 1   SeriousDlqin2yrs                      150000 non-null  int64  
 2   RevolvingUtilizationOfUnsecuredLines  150000 non-null  float64
 3   age                                   150000 non-null  int64  
 4   NumberOfTime30-59DaysPastDueNotWorse  150000 non-null  int64  
 5   DebtRatio                             150000 non-null  float64
 6   MonthlyIncome                         120269 non-null  float64
 7   NumberOfOpenCreditLinesAndLoans       150000 non-null  int64  
 8   NumberOfTimes90DaysLate               150000 non-null  int64  
 9   NumberRealEstateLoansOrLines          150000 non-null  int64  
 10  NumberOfTime60-89DaysPastDueNotWorse  150000 non-null  int64  
 11  

## 3. Gereksiz ID Kolonunun Kaldırılması

Veri setinde `Unnamed: 0` adında, yalnızca indeks bilgisi taşıyan ve 
müşteri hakkında anlamlı bilgi içermeyen bir ID kolonu bulunuyor. 
Modelleme sürecine katkısı olmadığı için bu kolonu kaldırıyoruz.

In [5]:
#ham veriye zarar vermemek için kopya üzerinde çalışıyoruz
df = df_raw.copy() 

if "Unnamed: 0" in df.columns:
    df = df.drop(columns=["Unnamed: 0"])

df.head(3)


,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0


### 4. Age Değerindeki Hataların Düzeltilmesi

EDA sırasında `age` değişkeninde 0 yaş değerine sahip tek bir kayıt tespit ettik.  0 yaş gerçekçi olmadığından veri giriş hatası olarak ele alıyorum.

Bu adımda:
- `age == 0` olan kaydı eksik (NaN) olarak işaretliyorum,
- ardından `age` sütunundaki eksikleri median yaş değeriyle dolduruyorum.

In [6]:
# Kaç adet age=0 var?
(df["age"] == 0).sum()


np.int64(1)

In [7]:
# 0 yaş gerçek bir değer olmadığından önce NaN olarak işaretlenir; 
# böylece median yalnızca gerçekçi yaş dağılımına göre hesaplanır.

df.loc[df["age"] == 0, "age"] = pd.NA
age_median = df["age"].median()
df["age"] = df["age"].fillna(age_median)

df["age"].describe()


count    150000.000000
mean         52.295553
std          14.771249
min          21.000000
25%          41.000000
50%          52.000000
75%          63.000000
max         109.000000
Name: age, dtype: float64

## 5. Eksik Değerlerin Genel Görünümü

Temizlikten sonra, hangi değişkende ne oranda eksik değer kaldığını 
tekrar kontrol ediyoruz. Özellikle `MonthlyIncome` ve `NumberOfDependents` 
değişkenlerindeki eksik oranları EDA’da detaylı incelemiştik.

In [8]:
df.isna().mean().sort_values(ascending=False)

MonthlyIncome                           0.198207
NumberOfDependents                      0.026160
SeriousDlqin2yrs                        0.000000
age                                     0.000000
RevolvingUtilizationOfUnsecuredLines    0.000000
DebtRatio                               0.000000
NumberOfTime30-59DaysPastDueNotWorse    0.000000
NumberOfOpenCreditLinesAndLoans         0.000000
NumberOfTimes90DaysLate                 0.000000
NumberRealEstateLoansOrLines            0.000000
NumberOfTime60-89DaysPastDueNotWorse    0.000000
dtype: float64

Bu çıktıda eksik değerlerin yalnızca iki değişkende yoğunlaştığı görülüyor:
- `MonthlyIncome` için oran ≈ %20
- `NumberOfDependents` için oran ≈ %2.6

Diğer tüm değişkenlerde eksik değer bulunmuyor.

## 6. Eksik Değerlerin Doldurulması (MonthlyIncome & NumberOfDependents)

EDA sonuçlarına göre:
- `MonthlyIncome` değişkeninde yaklaşık %20,
- `NumberOfDependents` değişkeninde yaklaşık %2.6 eksik değer bulunuyor.

Baseline için sade ve tekrarlanabilir bir yaklaşım olarak her iki değişkende de 
eksik değerleri median ile dolduruyoruz. 

Ayrıca `MonthlyIncome` için “gelir bilgisi eksik mi?” sorusunu yakalamak adına 
feature engineering aşamasında `IncomeMissing` isimli bir flag değişkeni eklenecek.


In [9]:
income_median = df["MonthlyIncome"].median()
df["MonthlyIncome"] = df["MonthlyIncome"].fillna(income_median)

dep_median = df["NumberOfDependents"].median()
df["NumberOfDependents"] = df["NumberOfDependents"].fillna(dep_median)

df.isna().mean().sort_values()


SeriousDlqin2yrs                        0.0
RevolvingUtilizationOfUnsecuredLines    0.0
age                                     0.0
NumberOfTime30-59DaysPastDueNotWorse    0.0
DebtRatio                               0.0
MonthlyIncome                           0.0
NumberOfOpenCreditLinesAndLoans         0.0
NumberOfTimes90DaysLate                 0.0
NumberRealEstateLoansOrLines            0.0
NumberOfTime60-89DaysPastDueNotWorse    0.0
NumberOfDependents                      0.0
dtype: float64

MonthlyIncome ve NumberOfDependents değişkenlerindeki eksikler, uç değerlerden etkilenmeyen median ile doldurulmuştur. Gelir değişkeni için ayrıca `IncomeMissing` flag’i feature engineering aşamasında eklenecektir.

## 7. Delinquency Değerlerindeki Aykırı Gözlemler

Üç delinquency değişkeninde (`NumberOfTime30-59DaysPastDueNotWorse`, 
`NumberOfTime60-89DaysPastDueNotWorse`, `NumberOfTimes90DaysLate`):

- Gözlemlerin büyük çoğunluğu 0 iken,
- Maksimum değer 98 olarak tekrar ediyor.

Bu değerlerin gerçek hayatta mümkün olmadığı ve sistematik bir hata olduğu sonucuna vardığımız için, bu değişkenlerde **üst sınır (capping)** uygulayarak değerleri 10 ile sınırlayacağız.
Burada 10 eşiği seçilmesinin sebebi, hem gerçek hayatta makul bir üst limit olması hem de yüksek gecikme sinyalini tamamen kaybetmeden 98 gibi hatalı değerleri törpülemesidir.

In [10]:
delinq_cols = [
    "NumberOfTime30-59DaysPastDueNotWorse",
    "NumberOfTime60-89DaysPastDueNotWorse",
    "NumberOfTimes90DaysLate",
]

for col in delinq_cols:
    max_before = df[col].max()
    df[col] = df[col].clip(upper=10)
    max_after = df[col].max()
    print(f"{col} | max before: {max_before}, max after: {max_after}")


NumberOfTime30-59DaysPastDueNotWorse | max before: 98, max after: 10
NumberOfTime60-89DaysPastDueNotWorse | max before: 98, max after: 10
NumberOfTimes90DaysLate | max before: 98, max after: 10


## 8. Temizlik Sonrası Özet İstatistikler

Temizlik adımlarından sonra temel değişkenlerin özet istatistiklerine tekrar bakarak 
herhangi bir beklenmedik durum olup olmadığını kontrol ediyoruz.


In [11]:
df.describe().T


,count,mean,std,min,25%,50%,75%,max
SeriousDlqin2yrs,150000.0,0.066840,0.249746,0.0,0.000000,0.000000,0.000000,1.0
RevolvingUtilizationOfUnsecuredLines,150000.0,6.048438,249.755371,0.0,0.029867,0.154181,0.559046,50708.0
age,150000.0,52.295553,14.771249,21.0,41.000000,52.000000,63.000000,109.0
NumberOfTime30-59DaysPastDueNotWorse,150000.0,0.263233,0.809436,0.0,0.000000,0.000000,0.000000,10.0
DebtRatio,150000.0,353.005076,2037.818523,0.0,0.175074,0.366508,0.868254,329664.0
MonthlyIncome,150000.0,6418.454920,12890.395542,0.0,3903.000000,5400.000000,7400.000000,3008750.0
NumberOfOpenCreditLinesAndLoans,150000.0,8.452760,5.145951,0.0,5.000000,8.000000,11.000000,58.0
NumberOfTimes90DaysLate,150000.0,0.107920,0.635481,0.0,0.000000,0.000000,0.000000,10.0
NumberRealEstateLoansOrLines,150000.0,1.018240,1.129771,0.0,0.000000,1.000000,2.000000,54.0
NumberOfTime60-89DaysPastDueNotWorse,150000.0,0.082633,0.534148,0.0,0.000000,0.000000,0.000000,10.0


- Delinquency değişkenlerinin maksimum değerleri artık 10 ile sınırlı.
- `age` değişkeni 21–95 aralığında.
- Eksik değer oranları tüm değişkenlerde sıfırlanmış durumda.

Bu tablo, uygulanan temizlik adımlarının beklediğimiz şekilde çalıştığını doğruluyor.

## 9. Age Değişkeninde Üst Limit (Capping)

Yaş dağılımında 95 yaş üzeri (96–109) aralığında yalnızca 63 gözlem bulundu ve bu değerler doğal bir yaş dağılımı gibi görünmüyor. Kredi skorlama uygulamalarında 95+ yaş aralığı çoğu zaman maskeleme veya sistemsel kodlama kaynaklı uç değer olarak değerlendirilir.

Bu nedenle `age` değişkenini 21–95 aralığında sınırlandırarak (winsorization) hem gerçekçi olmayan uç değerleri temizliyorum hem de modelin bu gözlemlerden gereksiz etkilenmesini engelliyorum.


In [12]:
# Age capping: 21–95 arası
df["age"] = df["age"].clip(lower=21, upper=95)
df["age"].describe()


count    150000.000000
mean         52.294093
std          14.766719
min          21.000000
25%          41.000000
50%          52.000000
75%          63.000000
max          95.000000
Name: age, dtype: float64

## 10. Temiz Verinin Kaydedilmesi

Artık temel temizlik adımları tamamlandı. 
Bu veri setini, sonraki aşamalarda (feature engineering ve baseline model) 
doğrudan kullanmak üzere diske kaydediyoruz.


In [13]:
df.to_csv(CLEAN_TRAIN, index=False)
print("Temiz veri kaydedildi:", CLEAN_TRAIN)


Temiz veri kaydedildi: C:\Users\YAĞMUR\Masaüstü\credit-risk-model\data\cs-training-clean.csv


## 11. Özet

Bu notebook'ta:

- ID kolonu kaldırıldı,
- `age` içindeki hatalı 0 değeri median ile düzeltildi,
- `MonthlyIncome` ve `NumberOfDependents` eksik değerleri median ile dolduruldu,
- Delinquency değişkenlerinde 98 gibi gerçekçi olmayan uç değerler 10 seviyesinde sınırlandırıldı,
- `age` değişkeni 21–95 aralığında üst/alt sınırla yeniden ölçeklendirildi,
- Temiz veri `cs-training-clean.csv` olarak kaydedildi.

Bir sonraki adımda bu temizlik kurallarını `src/data_preprocessing.py` içinde fonksiyon haline getirip baseline model kurulumunda yeniden kullanacağız.
